###  **СОДЕРЖАНИЕ**

1. ПРЕДПОДГОТОВКА ДАННЫХ, ПРЕДВАРИТЕЛЬНЫЙ АНАЛИЗ\
    1.1. Тестовая группа\
    1.2. Контрольные группы   
    1.3. Некоторые уточнения согласно условиям эксперимента
2. ПРОВЕРКА УСПЕШНОСТИ ЭКСПЕРИМЕНТА
3. ИМЕЕТ ЛИ НОВОВВЕДЕНИЕ СМЫСЛ СРЕДИ КАКИХ-ЛИБО КОНКРЕТНЫХ ГРУПП ПОЛЬЗОВАТЕЛЕЙ?\
    3.1. Пилотажный анализ проблемы\
    3.2. Основной блок аналитики
    
        3.2.1. Фактор "age" (возраст пользователей)
        3.2.2. Фактор "country" (страна пользователя)
        3.2.3. Фактор "views_count" (число полученных оценок)
        3.2.4. Фактор "attraction_coeff" (коэффициент привлекательности)
        3.2.5. Фактор "frequency" (частота посещений сайта)
        
4. ОБЩИЕ ВЫВОДЫ ПО ПРОЕКТУ

In [1]:
import numpy as np
import pandas as pd
from scipy.stats import norm, mannwhitneyu
import matplotlib.pyplot as plt
%matplotlib inline 
import seaborn as sns

from tqdm.auto import tqdm

from scipy import stats

plt.style.use('ggplot')

import statsmodels.api as sm
import statsmodels.formula.api as smf

from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from statsmodels.stats.multicomp import pairwise_tukeyhsd

from scipy.stats import kruskal
import scikit_posthocs as sp

In [2]:
sns.set(rc={'figure.figsize': (16,6)}, style='whitegrid') 

In [3]:

#          Согласно предоставленной информации, в эксперименте приняло участие три группы: тестовая (test),
#   контрольная 1 (control_1) и контрольная 2 (control_2). Судя по всему, имеем дело с классическим вариантом A/A/B теста.
#          Дополнительная контрольная А-ветка является страховкой (например, если в компании кто-то параллельно проводит 
#   эксперимент с той же целевой метрикой или чтобы убедиться в качестве сплитования). Встречается точка зрения,
#   что A/A/B тест - это компромиссный лайт-вариант процесса последовательного проведения A/A теста, а затем A/B - теста.
#          В  A/A/B тестах мы хотим принимать гипотезу H(0) в паре A1/A2 и отвергать H(0) на этапе A1+A2/B.


In [4]:

#   Предполагаемые:
#         Цель эксперимента: повысить выручку за счет изменения для новых пользователей из нескольких стран стоимости
#                            премиум-подписки при покупке через две новые платежные системы.  
#         Метрики: Retention, ARPPU, Active Users.
#         Гипотеза: ожидаем, что за счет изменения стоимости премиум-подписки будут получены достоверно значимые различия 
#                   по выручке между группами A1+A2/B.


In [5]:
#                                 1. ПРЕДПОДГОТОВКА ДАННЫХ, ПРЕДВАРИТЕЛЬНЫЙ АНАЛИЗ

In [6]:
users_test = pd.read_csv('https://stepik.org/media/attachments/lesson/409320/users_test.csv', encoding='Windows-1251', sep=';')

In [7]:
transactions_test = pd.read_csv('https://stepik.org/media/attachments/lesson/409320/transactions_test.csv', encoding='Windows-1251', sep=';', parse_dates=['joined_at', 'paid_at'])

In [8]:
users_control_1 = pd.read_csv('https://stepik.org/media/attachments/lesson/409320/users_control_1.csv', encoding='Windows-1251', sep=';')

In [9]:
transactions_control_1 = pd.read_csv('https://stepik.org/media/attachments/lesson/409320/transactions_control_1.csv', encoding='Windows-1251', sep=';', parse_dates=['joined_at', 'paid_at'])

In [10]:
users_control_2 = pd.read_csv('https://stepik.org/media/attachments/lesson/409320/users_control_2.csv', encoding='Windows-1251', sep=';')

In [11]:
transactions_control_2 = pd.read_csv('https://stepik.org/media/attachments/lesson/409320/transactions_control_2.csv', encoding='Windows-1251', sep=';', parse_dates=['joined_at', 'paid_at'])

HTTPError: HTTP Error 502: Bad Gateway

In [ ]:
#                                           1.1. Тестовая группа

In [ ]:
users_test.head()

In [ ]:
users_test.shape

In [ ]:
users_test.dtypes

In [ ]:
users_test.describe()

In [ ]:
users_test.isnull().sum()

In [ ]:
users_test.uid.nunique()  # все пользователи уникальны

In [ ]:
users_test.country.nunique()  # из какого количества стран

In [ ]:
users_test.query('total_revenue == 0').shape[0]  # из 4308 платежей 4162 равны 0.    

In [ ]:
#

In [ ]:
transactions_test.head()

In [ ]:
transactions_test.shape

In [ ]:
transactions_test.dtypes

In [ ]:
transactions_test.describe()

In [ ]:
transactions_test.isnull().sum()

In [ ]:
transactions_test.uid.nunique()  # не все пользователи уникальны

In [ ]:
transactions_test.uid.value_counts()     

In [ ]:
transactions_test.revenue.nunique() 

In [ ]:
transactions_test.describe(include='object')

In [ ]:
transactions_test.describe(include='datetime')

In [ ]:
transactions_test.product_type.unique()  

In [ ]:
transactions_test.groupby('product_type').agg({'revenue': 'sum'})

In [ ]:
 transactions_test.groupby('product_type').agg({'uid': 'count'})

In [ ]:
transactions_test.query('product_type == "premium_no_trial"').groupby('uid', as_index=False).agg({'revenue': 'count'})

In [ ]:
transactions_test.query('product_type == "premium_no_trial"').groupby('uid', as_index=False).agg({'revenue': 'sum'})

In [ ]:
# Объединяем данные
group_b_full = users_test.merge(transactions_test, how='right', on='uid')
group_b_full.head()

In [ ]:
group_b_full.shape

In [ ]:
group_b_full.info()

In [ ]:
product_type_vs_profit_b = group_b_full.groupby('product_type', as_index=False).total_revenue.sum()
product_type_vs_profit_b

In [ ]:
plt.figure(figsize=(10,5))
sns.barplot(data=product_type_vs_profit_b, x="product_type", y="total_revenue") 

In [ ]:
group_b_full.product_type.value_counts()

In [ ]:
group_b_full.uid.nunique()

In [ ]:
# Получаем срез пользователей с типом продукта 'premium_no_trial' 
group_b = group_b_full.query('product_type == "premium_no_trial"')
group_b.head()

In [ ]:
group_b.shape

In [ ]:
group_b.uid.nunique()  # уникальных ID в группе В только 67 штук

In [ ]:
group_b.uid.value_counts()

In [ ]:
group_b.info()

In [ ]:
group_b.groupby(['paid_at'], as_index=False).agg({'revenue': 'sum'}).revenue.plot()

In [ ]:
group_b.groupby(['paid_at'], as_index=False).agg({'total_revenue': 'sum'}).total_revenue.plot()

In [ ]:
sns.lineplot(x='paid_at', y ='revenue', hue='product_type', data=group_b)

In [ ]:
#                                         1.2. Контрольные группы

In [ ]:
users_control_1.head()

In [ ]:
users_control_1.shape

In [ ]:
users_control_1.info()

In [ ]:
users_control_1.describe()

In [ ]:
users_control_1.uid.nunique()      # все пользователи уникальны

In [ ]:
users_control_1.country.nunique()  # из какого количества стран

In [ ]:
users_control_1.query('total_revenue == 0').shape[0]  # Из 4340 платежей 4148 равны 0.   

In [ ]:
#

In [ ]:
transactions_control_1

In [ ]:
transactions_control_1.shape

In [ ]:
transactions_control_1.info()  # полный набор, без проблем, кроме "пустышек"

In [ ]:
# Убираем строки с пропущенными значениями  
transactions_control_1 = transactions_control_1.dropna(subset=['product_type'])

In [ ]:
transactions_control_1.head()

In [ ]:
transactions_control_1.shape

In [ ]:
transactions_control_1.info() 

In [ ]:
transactions_control_1.uid.nunique()  # не все пользователи уникальны

In [ ]:
transactions_control_1.uid.value_counts()

In [ ]:
transactions_control_1.describe(include='object')

In [ ]:
transactions_control_1.describe(include='datetime')

In [ ]:
transactions_control_1.groupby('product_type').agg({'revenue': 'sum'})

In [ ]:
 transactions_control_1.groupby('product_type', as_index=False).agg({'uid': 'count'})

In [ ]:
transactions_control_1.query('product_type == "premium_no_trial"').groupby('uid', as_index=False).agg({'revenue': 'count'})

In [ ]:
transactions_control_1.query('product_type == "premium_no_trial"').groupby('uid', as_index=False).agg({'revenue': 'sum'})

In [ ]:
# Объединяем данные
group_a1_full = users_control_1.merge(transactions_control_1, how='right', on='uid')
group_a1_full.head()

In [ ]:
group_a1_full.shape

In [ ]:
group_a1_full.info()

In [ ]:
product_type_vs_profit_a1 = group_a1_full.groupby('product_type', as_index=False).total_revenue.sum()
product_type_vs_profit_a1

In [ ]:
plt.figure(figsize=(10,5))
sns.barplot(data=product_type_vs_profit_a1, x="product_type", y="total_revenue")

In [ ]:
group_a1_full.product_type.value_counts()

In [ ]:
# Получаем срез пользователей с типом продукта 'premium_no_trial' 
group_a1 = group_a1_full.query('product_type == "premium_no_trial"')
group_a1.head()

In [ ]:
group_a1.shape

In [ ]:
group_a1.uid.nunique()

In [ ]:
group_a1.info()

In [ ]:
group_a1.groupby(['paid_at'], as_index=False).agg({'revenue': 'sum'}).revenue.plot()

In [ ]:
group_a1.groupby(['paid_at'], as_index=False).agg({'total_revenue': 'sum'}).total_revenue.plot()

In [ ]:
sns.lineplot(x='paid_at', y='revenue', hue='product_type', data=group_a1)

In [ ]:
#

In [ ]:
users_control_2.head()

In [ ]:
users_control_2.shape

In [ ]:
users_control_2.info()

In [ ]:
users_control_2.describe()

In [ ]:
users_control_2.uid.nunique()  # все пользователи уникальны

In [ ]:
users_control_2.country.nunique()  # из какого количества стран

In [ ]:
users_control_2.query('total_revenue == 0').shape[0]  # из 4264 платежей 4077 равны 0.    

In [ ]:
#

In [ ]:
transactions_control_2.head()

In [ ]:
transactions_control_2.shape

In [ ]:
transactions_control_2.info()  # полный df, без проблем 

In [ ]:
transactions_control_2.uid.nunique()  # не все пользователи уникальны

In [ ]:
transactions_control_2.uid.value_counts()

In [ ]:
transactions_control_2.describe(include='object')

In [ ]:
transactions_control_2.describe(include='datetime')

In [ ]:
 transactions_control_2.groupby('product_type').agg({'revenue': 'sum'})

In [ ]:
 transactions_control_2.groupby('product_type').agg({'uid': 'count'})

In [ ]:
transactions_control_2.query('product_type == "premium_no_trial"').groupby('uid', as_index=False).agg({'revenue': 'count'})

In [ ]:
transactions_control_2.query('product_type == "premium_no_trial"').groupby('uid', as_index=False).agg({'revenue': 'sum'})

In [ ]:
# Объединяем данные 
group_a2_full = users_control_2.merge(transactions_control_2, how='right', on='uid')
group_a2_full

In [ ]:
group_a2_full.shape

In [ ]:
group_a2_full.info()

In [ ]:
product_type_vs_profit_a2 = group_a2_full.groupby('product_type', as_index=False).total_revenue.sum()
product_type_vs_profit_a2

In [ ]:
plt.figure(figsize=(10,5))
sns.barplot(data=product_type_vs_profit_a2, x="product_type", y="total_revenue") 

In [ ]:
group_a2_full.product_type.value_counts()

In [ ]:
group_a2_full.uid.nunique()

In [ ]:
# Получаем срез пользователей с типом продукта 'premium_no_trial' 
group_a2 = group_a2_full.query('product_type == "premium_no_trial"')
group_a2.head()

In [ ]:
group_a2.shape

In [ ]:
group_a2.info()

In [ ]:
group_a2.uid.nunique()

In [ ]:
group_a2.groupby(['paid_at'], as_index=False).agg({'revenue': 'sum'}).revenue.plot()

In [ ]:
group_a2.groupby(['paid_at'], as_index=False).agg({'total_revenue': 'sum'}).total_revenue.plot()

In [ ]:
sns.lineplot(x='paid_at', y='revenue', hue='product_type', data=group_a2)

In [ ]:
#                                  1.3. Некоторые уточнения согласно условиям эксперимента.

In [ ]:
#   Подразумевается, что эксперимент проводился в один и тот же временной период для всех групп. 
#   Т.е. время начала и конца эксперимента в группах А1,А2,В должно быть синхронизированно. 
#   Что это за интервал времени?
#                      c '2017-02-11 00:13:00' по '2017-11-14 16:50:00'

In [ ]:
group_b.paid_at.min()

In [ ]:
group_b.paid_at.max()

In [ ]:
group_a1.paid_at.min()

In [ ]:
group_a1.paid_at.max()

In [ ]:
group_a2.paid_at.min()

In [ ]:
group_a2.paid_at.max()

In [ ]:
lower_edge_time = max(group_b.paid_at.min(), group_a1.paid_at.min(), group_a2.paid_at.min())

In [ ]:
lower_edge_time

In [ ]:
upper_edge_time = min(group_b.paid_at.max(), group_a1.paid_at.max(), group_a2.paid_at.max())

In [ ]:
upper_edge_time

In [ ]:
# Приводим группы к временным границам эксперимента
group_b = group_b.query('(paid_at >= "2017-02-11 00:13:00") & (paid_at <= "2017-11-14 16:50:00")')
group_a1 = group_a1.query('(paid_at >= "2017-02-11 00:13:00") & (paid_at <= "2017-11-14 16:50:00")')
group_a2 = group_a2.query('(paid_at >= "2017-02-11 00:13:00") & (paid_at <= "2017-11-14 16:50:00")')

In [ ]:

#     Видим, что количество стран в контрольных группах А1,А2 одинаково; набор стран в группах А1,А2 несколько разнится;
#  пользователи между группами А1,А2 распределены не совсем равномерно и корректно - а ведь это должно являться важным
#  начальным условием для А/А теста.
#     Большой вопрос вызывает набор стран в тестовой группе В (на 1 страну меньше; выбраны несколько других стран; количество
#  пользователей из каждой страны не совпадает с аналогичным набором из контрольных групп А1,А2).
#     Кроме того, начальные, "сырые" данные говорят о 273 пользователях из группы "В", 377 из группы "А1", 328 из группы "А2".  
#     Создается общее впечатление, что эксперимент организован "нечисто", сплитование не совсем корректно,
#  вероятность ошибки возрастает. 


In [ ]:
group_b.country_x.nunique()

In [ ]:
group_b.country_x.value_counts()

In [ ]:
group_a1.country_x.nunique()

In [ ]:
group_a1.country_x.value_counts()

In [ ]:
group_a2.country_x.nunique()

In [ ]:
group_a2.country_x.value_counts()

In [ ]:
# количество участников (со всеми типами подписки)

In [ ]:
full_0 = (users_test.uid + transactions_test.uid).dropna().drop_duplicates()           # группа "В" перед началом эксперимента
full_0.shape[0]

In [ ]:
full_1 = (users_control_1.uid + transactions_control_1.uid).dropna().drop_duplicates() # группа "А1" перед началом эксперимента
full_1.shape[0]

In [ ]:
full_2 = (users_control_2.uid + transactions_control_2.uid).dropna().drop_duplicates()  #группа "А2" перед началом эксперимента
full_2.shape[0]

In [ ]:
#                                          2. ПРОВЕРКА УСПЕШНОСТИ ЭКСПЕРИМЕНТА 

In [ ]:
data_column_a1 = group_a1['revenue'].to_frame()
data_column_a2 = group_a2['revenue'].to_frame()
data_column_b = group_b['revenue'].to_frame() 

In [ ]:

#   Проверим группы на нормальность распределения метрики 'revenue' внутри группы, используя тест Шапиро-Уилка.
#   Нулевая гипотеза H(0) в случае Шапиро-Уилка - нормальность распределения метрики.

#   Видим, что во всех случаях p<0.05, поэтому отклоняем H(0), распределение отличается от нормального - в условиях небольших
#   выборок это исключает возможность применения T-критерия Стьюдента.


In [ ]:
stats.shapiro(data_column_a1)

In [ ]:
stats.shapiro(data_column_a2)

In [ ]:
stats.shapiro(data_column_b)

In [ ]:
#

In [ ]:

#   Проверяем, сходится ли А1\A2 тест; используем непараметрический тест – U-критерий Манна-Уитни.
#   Видим, что p>0.05, не можем отклонить гипотезу H(0) в паре A1/A2 - различий по метрике 'revenue' между группами нет.  


In [ ]:
mannwhitneyu(data_column_a1, data_column_a2)  

In [ ]:
#   Было бы интересно проверить сходимость А1\A2 теста по другому:

In [ ]:
n = 104
simulations = 1000
n_s = 30
res = []

# Запуск симуляций A/A теста
for i in tqdm(range(simulations)):
    s1 = group_a1['revenue'].sample(n_s, replace = False).values
    s2 = group_a2['revenue'].sample(n_s, replace = False).values
    res.append(stats.ttest_ind(s1, s2, equal_var = False)[1]) # сохраняем pvalue

plt.hist(res, bins = 50)
plt.style.use('ggplot')
plt.xlabel('pvalues')
plt.ylabel('frequency')
plt.title("Histogram of ttest A/A simulations ")
plt.show()

# Проверяем, что количество ложноположительных случаев не превышает альфа
sum(np.array(res) <0.05) / simulations

In [ ]:
#   Видим, что FPR < альфа=0.05
#   A/A тест по метрике 'revenue' сходится, все нормально.

In [ ]:
#

In [ ]:
# Переходим к анализу A1+A2/B теста

In [ ]:
data_column_a1.describe()

In [ ]:
data_column_a2.describe()

In [ ]:
data_column_b.describe()

In [ ]:
data_column_a1_a2 = pd.concat([group_a1['revenue'], group_a2['revenue']], ignore_index=True).to_frame()
data_column_a1_a2 = data_column_a1_a2.astype('int64')
data_column_a1_a2

In [ ]:
data_column_a1_a2.max()

In [ ]:
data_column_a1_a2.min()

In [ ]:
data_column_a1_a2.plot()

In [ ]:
data_column_b.max()

In [ ]:
data_column_b.min()

In [ ]:
data_column_b.plot()

In [ ]:

#    Применяем непараметрику Манна-Уитни.
#    Получаем p<0.05, что позволяет нам отвергнуть гипотезу H(0),
#    различия между тестовой группой В и контрольной A1+A2 по метрике 'revenue' достоверны. 


In [ ]:
mannwhitneyu(data_column_a1_a2, data_column_b) 

In [ ]:
# Было бы интересно проверить то же самое с помощью bootstrap:

In [ ]:
def get_bootstrap(
    data_column_1,
    data_column_2, 
    boot_it = 1000, 
    statistic = np.mean,
    bootstrap_conf_level = 0.95 
):
    boot_len = max([len(data_column_1), len(data_column_2)])
    boot_data = []
    for i in tqdm(range(boot_it)): 
        samples_1 = data_column_1.sample(
            boot_len, 
            replace = True 
        ).values
        
        samples_2 = data_column_2.sample(
            boot_len, 
            replace = True
        ).values
        
        boot_data.append(statistic(samples_1-samples_2)) # mean() - применяем статистику
        
    pd_boot_data = pd.DataFrame(boot_data)
        
    left_quant = (1 - bootstrap_conf_level)/2
    right_quant = 1 - (1 - bootstrap_conf_level) / 2
    quants = pd_boot_data.quantile([left_quant, right_quant])
        
    p_1 = norm.cdf(
        x = 0, 
        loc = np.mean(boot_data), 
        scale = np.std(boot_data)
    )
    p_2 = norm.cdf(
        x = 0, 
        loc = -np.mean(boot_data), 
        scale = np.std(boot_data)
    )
    p_value = min(p_1, p_2) * 2
        
    # Визуализация
    _, _, bars = plt.hist(pd_boot_data[0], bins = 50)
    for bar in bars:
        if abs(bar.get_x()) <= quants.iloc[0][0] or abs(bar.get_x()) >= quants.iloc[1][0]:
            bar.set_facecolor('red')
        else: 
            bar.set_facecolor('grey')
            bar.set_edgecolor('black')
    
    plt.style.use('ggplot')
    plt.vlines(quants,ymin=0,ymax=50,linestyle='--')
    plt.xlabel('boot_data')
    plt.ylabel('frequency')
    plt.title("Histogram of boot_data")
    plt.show()
       
    return {"boot_data": boot_data, 
            "quants": quants, 
            "p_value": p_value}

In [ ]:
sample_1 = data_column_b
sample_2 = data_column_a1_a2

In [ ]:
booted_data = get_bootstrap(sample_1, sample_2) 

In [ ]:
booted_data["quants"]  # ДИ

In [ ]:
booted_data["p_value"]  # альфа

In [ ]:

# Видим, что в случае применения bootstrap с параметром распределения mean, значение "0" не попадает в ДИ, P<0.05, 
# как следствие, можем отвергнуть гипотезу H(0) о том, что нет различий между контрольной и тестовой группой
# по метрике 'revenue'. Можем принять гипотезу H(1) - различия между группами по метрике 'revenue' - достоверны.


In [ ]:

#  ВЫВОД:
#      Таким образом, можем констатировать, что эксперимент в целом был успешен. Есть вопросы к планированию эксперимента
#  (некорректное сплитование на группы), что может поставить под вопрос ценность полученных данных.

#      Проанализировав некоторыё основные метрики (Revenue, ARPPU, Active Users) можно увидеть, что за период эксперимента
#  в тестовой группе "В" размер Revenue стал несколько меньше, чем в группе "А1", но больше чем в "А2";  
#  количество Active Users в тестовой группе "В" упало по сравнению с "А1" и "А2" (можно предположить, что они ушли с
#  премиум-подписки в связи с изменившимися тарифами); в то же время за счет изменения стоимости премиум-подписки и
#  сокращения числа пользователей, в тестовой группе "В", по сравнению с группами "А1" и "А2", ощутимо выросла метрика ARPPU.


In [ ]:
# Revenue по сегменту "premium_no_trial"

In [ ]:
Revenue_b = group_b.total_revenue.sum()      
Revenue_b

In [ ]:
Revenue_a1 = group_a1.total_revenue.sum()    
Revenue_a1

In [ ]:
Revenue_a2 = group_a2.total_revenue.sum()    
Revenue_a2

In [ ]:
# Revenue (все типы подписки) 

In [ ]:
Revenue_b = group_b_full.total_revenue.sum()  
Revenue_b

In [ ]:
Revenue_a1 = group_a1_full.total_revenue.sum()    
Revenue_a1

In [ ]:
Revenue_a2 = group_a2_full.total_revenue.sum()   
Revenue_a2

In [ ]:
# ARPPU

In [ ]:
ARPPU_b = group_b.total_revenue.sum() / group_b.uid.count()     # ARPPU тестовой группы "В" по сегменту "premium_no_trial"
ARPPU_b

In [ ]:
ARPPU_a1 = group_a1.total_revenue.sum() / group_a1.uid.count()  # ARPPU тестовой группы "A1" по сегменту "premium_no_trial"
ARPPU_a1

In [ ]:
ARPPU_a2 = group_a2.total_revenue.sum() / group_a2.uid.count()   # ARPPU тестовой группы "A2" по сегменту "premium_no_trial"
ARPPU_a2

In [ ]:
# Active Users (сегмент "premium_no_trial")

In [ ]:
group_b.uid.nunique()

In [ ]:
group_a1.uid.nunique()

In [ ]:
group_a2.uid.nunique()

In [ ]:
# Active Users (все типы подписки)

In [ ]:
group_b_full.uid.nunique()

In [ ]:
group_a1_full.uid.nunique()

In [ ]:
group_a2_full.uid.nunique()

In [ ]:
#                       3. ИМЕЕТ ЛИ НОВОВВЕДЕНИЕ СМЫСЛ СРЕДИ КАКИХ-ЛИБО КОНКРЕТНЫХ ГРУПП ПОЛЬЗОВАТЕЛЕЙ

In [ ]:
#                                            3.1. Пилотажный анализ проблемы

In [ ]:

#      Что значит "нововведение имеет смысл среди каких-либо конкретных групп пользователей"? Речь идет об изменении
#   стоимости премиум-подписки для каких-то конкретных групп пользователей. Т.е. надо выявить силу и характер взаимосвязи между
#   какими-то переменными, одна из которых - нормированная выручка (ЗП), при условии что product_type = "premium" 
#   (в варианте no_trial).

#      Интересно, кто они, пользователи, оплачивающие тариф "premium_no_trial"? За что готовы платить, каковы их предпочтения
#   или может быть какие-то отличительные признаки? Есть смысл посмотреть на это сначало на одной из контрольных, "чистых"
#   групп, ещё не подвергшейся экспериментальному воздействию.    
    
#      Логичным является предположить, что группы пользователей, среди которых есть смысл делать нововведение,
#   могут быть обнаружены в процессе работы, например, со следующими категориями предоставленных данных:
#       -age (возраст) - вероятно пользователи возраста 20-30 лет, более мотивированы на знакомства, чем в 50-60 лет;
#       -coins (внутренняя валюта) - раз люди вложились во внутреннюю валюту, они наверное заинтересованы в присутствии на сайте;
#       -views_count (число полученных оценок) - чаще всего лидеры, активные пользователи сайта, в "топе"; 
#       -country (страна) - страны различаются по уровню жизни, экономического развитию, традициям, 
#                                                                                                     и т.д.


In [ ]:
group_a2.head()

In [ ]:
(group_a2.country_x ==  group_a2.country_y).unique()

In [ ]:
group_a2 = group_a2.drop(['country_y'], axis=1)  # убираем дубликат колонки "country_x"

In [ ]:
group_a2['frequency'] = (group_a2['visit_days'].str.count(',') + 1)  # создаем новую колонку "frequency" на основе "visit_days"

In [ ]:
group_a2 = group_a2.fillna({'coins': group_a2.coins.median()})  # убираем NaN в 'coins'

In [ ]:
group_a2.shape

In [ ]:
group_a2.isnull().sum()

In [ ]:
group_a2 = group_a2.fillna({'frequency': group_a2.frequency.median()})  # убираем NaN в 'frequency'

In [ ]:

#    Прибегнем к визуализации, например диаграммы рассеяния неплохо подходят для исследования зависимости между переменными;
# возможно это поможет нам определиться, какие факторы можно (нужно) взять для дальнейшей работы.


In [ ]:
sns.set(style='whitegrid', rc={'figure.figsize': (10,5)})
sns.scatterplot(x = 'age', y = 'total_revenue', data = group_a2)
plt.title('Взаимосвязь возроста пользователя и выручки')
plt.xlabel('Возраст пользователя')
plt.ylabel('Нормированная выручка')

In [ ]:
sns.set(style='whitegrid', rc={'figure.figsize': (10,5)})
sns.lmplot(x='age', y='total_revenue', hue='gender', data=group_a2)  # с добавлением группирующей переменной 'gender'

In [ ]:
group_b_full.age.describe()

In [ ]:
group_a1_full.age.describe()

In [ ]:
group_a2_full.age.describe()

In [ ]:
# Предварительное заключение: 'age' - интересный фактор влияния, особенно если разбить на возрастные подгруппы 

In [ ]:
#

In [ ]:
sns.set(style='whitegrid', rc={'figure.figsize': (10,5)})
sns.lmplot(x='gender', y='total_revenue', hue='gender', data=group_a2)

In [ ]:
group_a2_full.gender.value_counts() 

In [ ]:
group_a1_full.gender.value_counts() 

In [ ]:
group_b_full.gender.value_counts() 

In [ ]:
# Предварительное заключение: представителей "gender=0" (скорее всего это женщины) очень мало, основная масса пользователей
# имеют "gender=1". Фактор "gender" не очень интересен для дальнейшего анализа.

In [ ]:
#

In [ ]:
sns.lmplot(x='coins', y='total_revenue', data=group_a2)

In [ ]:
group_b_full.info()

In [ ]:
group_a1_full.info()

In [ ]:
group_a2_full.info()

In [ ]:

# Предварительное заключение: фактор "coins" ('внутренняя валюта') кажется в общем-то вполне приемлемым для анализа,
# но взглянув на "coins" в группах "group_A1_full", "group_A2_full", "group_B_full", видим большое количество NaN.
# Теоретически можно все пропущенные значения заполнить медианными значениями "coins" по данной группе, но как это
# скажется на результатах - не совсем понятно. Отказываемся от анализа по фактору "coins".


In [ ]:
#

In [ ]:
sns.set(style='whitegrid', rc={'figure.figsize': (10,5)})
sns.lmplot(x='frequency', y='total_revenue', data=group_a2)

In [ ]:
# Предварительное заключение: фактор 'frequency' может быть интересен для дальнейшей работы.

In [ ]:
#

In [ ]:
sns.set(style='whitegrid', rc={'figure.figsize': (10,5)})
sns.lmplot(x='attraction_coeff', y='total_revenue', data=group_a2)

In [ ]:
sns.set(style='whitegrid', rc={'figure.figsize': (10,5)})
sns.lmplot(x='attraction_coeff', y='total_revenue', data=group_a2.query('attraction_coeff <= 600'))

In [ ]:
# Предварительное заключение: фактор 'attraction_coeff' может быть интересен для дальнейшей работы, если взять срез " < 600 " 

In [ ]:
#

In [ ]:
sns.set(style='whitegrid', rc={'figure.figsize': (10,5)})
sns.lmplot(x='age_filter_start', y='total_revenue', data=group_a2)

In [ ]:
# Предварительное заключение: фактор "age_filter_start" не очень интересен для дальнейшего анализа. 

In [ ]:
#

In [ ]:
sns.set(style='whitegrid', rc={'figure.figsize': (10,5)})
sns.lmplot(x='age_filter_end', y='total_revenue', data=group_a2)

In [ ]:
# Предварительное заключение: фактор "age_filter_end" не очень интересен для дальнейшего анализа.

In [ ]:
#

In [ ]:
sns.set(style='whitegrid', rc={'figure.figsize': (10,5)})
sns.lmplot(x='views_count', y='total_revenue', data=group_a2)

In [ ]:
sns.set(style='whitegrid', rc={'figure.figsize': (10,5)})
sns.lmplot(x='views_count', y='total_revenue', data=group_a2.query('gender == 1 & views_count < 400'))

In [ ]:
# Предварительное заключение: фактор 'views_count' может быть интересен для дальнейшего анализа.

In [ ]:

#  Кроме того, для дальнейшей работы может быть интересен фактор 'country_x', эффективность работы с которым была доказана выше
#  (однако остается вопрос: так для каких именно стран стоит делать нововведение?)


In [ ]:
#

In [ ]:

#       Попробуем получить какую-то информацию, рассматривая корреляционную связь между разными факторами.
#       Коэффициент корреляции в статистическом смысле обозначает силу и характер взаимосвязи между двумя
#  количественными переменными.
#       Посмотрим коэффициенты корреляции по Пирсону и по Спирмену. Будем ориентироваться на расчеты по Спирмену (коэфф.
#  корреляции Пирсона очень чувствителен к выбросам, асимметрии, бимодальности).
#       "По Спирмену":  можем утверждать (используя градацию шкалы Чеддока), что у фактора "total_revenue" 
#  выявлена 'умеренная' корреляционная связь с фактором "views_count" (+0.42) (т.е. число полученных оценок у пользователей),
#  а также 'слабая' корреляционная связь с фактором "frequency" (+0.28) и age (+0.12). 
#       Интересно, что корреляционная связь с фактором "frequency" проявляется и по Спирмену, и по Пирсону.  
#       Отметим, что сила и характер взаимосвязи между исследуемыми элементами еще не обязательно говорит о причинно-
#  следственной зависимости. Но корреляция МОЖЕТ означать причинно-следственную зависимость.


In [ ]:
# Формируем набор независимых переменных (НЗ)
cols_to_keep = ['age', 'attraction_coeff', 'coins', 'country_x', 'views_count', 'frequency', 'total_revenue']

In [ ]:
df = group_a2[cols_to_keep]
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
df.dtypes

In [ ]:
df.corr()  # по Пирсону

In [ ]:
df.corr()['total_revenue'].sort_values(ascending=False).round(2)

In [ ]:
df.corr(method='spearman')  # по Спирмену

In [ ]:
df.corr(method='spearman')['total_revenue'].sort_values(ascending=False).round(2)

In [ ]:
#                                       3.2. Основной блок аналитики

In [ ]:
#                                 3.2.1. Фактор "age" (возраст пользователей)

In [ ]:
# Распределим всех пользователей по возрастным группам

cut_labels = ['16-20', '21-25', '26-30', '31-35', '36-40', '41-45', '46-50', '51-55', '56-99']
cut_bins = [15, 20, 25, 30, 35, 40, 45, 50, 55, 99]

group_b_full = group_b_full.assign(age_diff = pd.cut(group_b_full['age'], bins=cut_bins, labels=cut_labels))
group_a1_full = group_a1_full.assign(age_diff = pd.cut(group_a1_full['age'], bins=cut_bins, labels=cut_labels))
group_a2_full = group_a2_full.assign(age_diff = pd.cut(group_a2_full['age'], bins=cut_bins, labels=cut_labels))

In [ ]:
group_b_full['group'] = 'B'
group_a1_full['group'] = 'A'
group_a2_full['group'] = 'A'       

In [ ]:
all_groups = pd.concat([group_b_full, group_a1_full, group_a2_full], ignore_index=True, verify_integrity=True)

In [ ]:
(all_groups.country_x == all_groups.country_y).unique()

In [ ]:
all_groups = all_groups.drop(['country_y'], axis=1)  # убираем дубликат колонки "country_x"

In [ ]:
all_groups.head()

In [ ]:
work_with_age = pd.DataFrame(columns=['group_test', 'group_control', 'age_diff', 'p_alpha', 'p_value', 'result'])

answer = ["Отвергаем гипотезу H(0)", "Не можем отвергнуть гипотезу H(0)"]

for i in all_groups['age_diff'].unique():
    
    s1 = all_groups.query('group == "B" & age_diff in @i')['total_revenue']
    s2 = all_groups.query('group != "B" & age_diff in @i')['total_revenue']
    
    p_value_result = mannwhitneyu(s1, s2)[1]
    
    new_row = {'group_test': 'B',
               'group_control': 'A',
               'age_diff': i,
               'p_alpha': 0.05 / ((len(cut_labels) * (len(cut_labels)-1)) / 2),  # we would do it using Bonferroni correction
               'p_value': p_value_result,
               'result': answer[p_value_result >= (0.05 / ((len(cut_labels) * (len(cut_labels)-1)) / 2))]}
    
    work_with_age = work_with_age.append([new_row])

work_with_age

In [ ]:

#    Вывод: отвергнута гипотеза H(0) в отношении групп пользователей 16-20 лет, 56-99 лет (фактор "age"), что позволяет нам  
# говорить о статистически значимых различиях между тестовой группой В и контрольной A1+A2 по метрике 'total_revenue' (у 
# пользователей 16-20, 56-99 лет).


In [ ]:
#                                     3.2.2. Фактор "country" (страна пользователя)

In [ ]:
group_b.country_x.unique()  # будем ориентироваться на список стран из тестовой (экспериментальной) группы "В"

In [ ]:
group_b.country_x.nunique()

In [ ]:
all_groups.query('group == "B" & product_type == "premium_no_trial"').country_x.unique()  # хорошо, то же самое что в группе "В"

In [ ]:
all_groups.query('group != "B" & product_type == "premium_no_trial"').country_x.unique()

In [ ]:
all_groups.country_x.isin(['United States of America', 'Canada', 'Spain', 'Latvia', 'Italy',
       'United Kingdom (Great Britain)', 'France', 'Argentina',
       'United Arab Emirates', 'Chile', 'Israel', 'Australia', 'Mexico',
       'Germany', 'Turkey', 'Belgium']).value_counts()

In [ ]:
new_list = all_groups.country_x.isin(['United States of America', 'Canada', 'Spain', 'Latvia', 'Italy',
       'United Kingdom (Great Britain)', 'France', 'Argentina',
       'United Arab Emirates', 'Chile', 'Israel', 'Australia', 'Mexico',
       'Germany', 'Turkey', 'Belgium'])

In [ ]:
all_groups_0 = all_groups[new_list]
all_groups_0.head() 

In [ ]:
group_b.country_x.value_counts()

In [ ]:
# Исключим из списка стран также те, от которых только по 1 представителю - вряд ли корректно делать выводы на основе 1 наблюдения

all_groups_0 = all_groups_0.query('country_x != "Latvia" & country_x != "Australia" & country_x != "Mexico" & country_x != "Turkey" & country_x != "Belgium"')

In [ ]:
work_with_country = pd.DataFrame(columns=['group_test', 'group_control', 'country', 'p_alpha', 'p_value', 'result'])

answer = ["Отвергаем гипотезу H(0)", "Не можем отвергнуть гипотезу H(0)"]

for i in all_groups_0['country_x'].unique():
    
    s1 = all_groups_0.query('group == "B" & country_x in @i & product_type == "premium_no_trial"')['total_revenue']
    s2 = all_groups_0.query('group != "B" & country_x in @i & product_type == "premium_no_trial"')['total_revenue']
    
    p_value_result = mannwhitneyu(s1, s2)[1]
    
    new_row = {'group_test': 'B',
               'group_control': 'A',
               'country': i,
               'p_alpha': 0.05 / group_b.country_x.nunique(),  # we would do it using Bonferroni correction
               'p_value': p_value_result,
               'result': answer[p_value_result >= (0.05 / group_b.country_x.nunique())]}
    
    work_with_country = work_with_country.append([new_row])

work_with_country

In [ ]:

#    Вывод: отвергнута гипотеза H(0) в отношении группы пользователей из страны "United States of America" (фактор "country"),
# что позволяет нам говорить о статистически значимых различиях (по метрике 'total_revenue') между пользователями из 
# тестовой группы В и контрольной A1+A2 (из страны "United States of America").


In [ ]:

# Далее прибегнем к ANOVA. Дисперсии внутри наших групп должны быть примерно одинаковы, проверим это с помощью критерия Левена. 
# Видим, что p>0.05, гипотезу H(0) отклонить не можем, дисперсия гомогенна в каждой из групп.
# Размеры выборок достаточно велики (рассматриваем все типы подписки), применение дисперсионного анализа возможно. 


In [ ]:
k = all_groups.query('group == "A"').total_revenue
l = all_groups.query('group == "B"').total_revenue

In [ ]:
stats.levene(k, l)

In [ ]:
#                                      3.2.3. Фактор "views_count" (число полученных оценок)

In [ ]:
all_groups_1 = all_groups.query('gender == 1 & views_count < 400')

In [ ]:
all_groups_1.views_count.quantile(q=[.33, .66])

In [ ]:
all_groups_1.views_count.describe()

In [ ]:
all_groups_1.groupby(['group', 'views_count'], as_index=False).agg({'total_revenue': 'mean'})

In [ ]:
df_1 = all_groups_1.groupby(['group', 'views_count'], as_index=False).agg({'total_revenue': 'mean'})

In [ ]:
cut_labels_1 = ['0-32', '33-112', '113-364']
cut_bins_1 = [-1, 32, 112, 364]

df_1 = df_1.assign(views_count_diff = pd.cut(df_1['views_count'], bins=cut_bins_1, labels=cut_labels_1))

In [ ]:
formula = 'total_revenue ~ group + views_count_diff + group:views_count_diff'
model = ols(formula, df_1).fit()
aov_table = anova_lm(model, typ=2)
print(aov_table)

In [ ]:
# Как видим, эффект от влияния обоих факторов на ЗП ('total_revenue') является статистически значимым (в отличии от их взаимодействия)

In [ ]:
df_1.views_count_diff = df_1.views_count_diff.astype(object)

In [ ]:
df_1['combination'] = df_1['group'] + ' \ ' + df_1['views_count_diff'] 

In [ ]:
print(pairwise_tukeyhsd(df_1['total_revenue'], groups=df_1['combination']).summary())

In [ ]:

# ВЫВОД: видим, что гипотезы H(0) были отклонены при сравнении средних значений в парах A\0-32 и B\113-364,
# A\113-364 и B\113-364, A\33-112 и B\113-364, A\33-112 и B\113-364, B\0-32 и B\113-364, что позволяет нам говорить о
# статистически значимых различиях (по метрике 'total_revenue') между пользователями из тестовой группы В и
# контрольной A1+A2 (показатели статистически значимо увеличились).   


In [ ]:
# Проверим, что получим при использовании непараметрических тестов (Kruskal-Wallis Test)

data1 = df_1.query('group == "A" & views_count_diff == "0-32"').total_revenue 
data2 = df_1.query('group == "A" & views_count_diff == "33-112"').total_revenue
data3 = df_1.query('group == "A" & views_count_diff == "113-364"').total_revenue

data4 = df_1.query('group == "B" & views_count_diff == "0-32"').total_revenue 
data5 = df_1.query('group == "B" & views_count_diff == "33-112"').total_revenue
data6 = df_1.query('group == "B" & views_count_diff == "113-364"').total_revenue

In [ ]:
stat, p = kruskal(data1, data2, data3, data4, data5, data6)
print('Statistics=%.3f, p=%.3f' % (stat, p)) #  отвергли гипотезу H(0)

In [ ]:
data = [data1, data2, data3, data4, data5, data6]

sp.posthoc_dunn(data, p_adjust = 'bonferroni') #  perform Dunn's test using a Bonferonni correction for the p-values

In [ ]:
sp.posthoc_dunn(data, p_adjust = 'holm') #  perform Dunn's test using a Holm correction for the p-values

In [ ]:

# ВЫВОД: видим, что при использовании непараметрических Kruskal-Wallis test и Dunn test, гипотезы H(0) были отклонены при
# сравнении значений в парах A\0-32 и B\113-364, A\33-112 и B\113-364, что позволяет нам говорить о статистически значимых
# различиях (по метрике 'total_revenue') между пользователями из тестовой группы В и контрольной A1+A2. 

# Интресно, что и с поправкой Бонферрони, и с более "мягкой" поправкой Холма получаем одинаковые конечные результаты.

# Принимаем решение в конечных выводах опираться на "непараметрику" (учитывая более строгий характер поправки Бонферрони;
# учитывая характеристики исходных групп). 


In [ ]:
#                                   3.2.4. Фактор "attraction_coeff" (коэффициент привлекательности)

In [ ]:
all_groups_2 = all_groups.query('attraction_coeff < 600')

In [ ]:
all_groups_2.attraction_coeff.quantile([.33, .66])

In [ ]:
all_groups_2.attraction_coeff.describe()

In [ ]:
all_groups_2.groupby(['group', 'attraction_coeff'], as_index=False).agg({'total_revenue': 'mean'})

In [ ]:
df_2 = all_groups_2.groupby(['group', 'attraction_coeff'], as_index=False).agg({'total_revenue': 'mean'})

In [ ]:
cut_labels_2 = ['0-199', '200-328', '329-594']
cut_bins_2 = [-1, 199, 328, 594]

df_2 = df_2.assign(attraction_coeff_diff = pd.cut(df_2['attraction_coeff'], bins=cut_bins_2, labels=cut_labels_2))

In [ ]:
formula = 'total_revenue ~ group + attraction_coeff_diff + group:attraction_coeff_diff'

model = ols(formula, df_2).fit()
aov_table = anova_lm(model, typ=2)
print(aov_table)

In [ ]:

# ВЫВОД: Как видим, эффект от влияния обоих факторов на ЗП ('total_revenue') статистически незначим (как и их взаимодействие) 


In [ ]:
# Проверим, что получим при использовании непараметрических тестов (Kruskal-Wallis Test)

data_1 = df_2.query('group == "A" & attraction_coeff_diff == "0-199"').total_revenue 
data_2 = df_2.query('group == "A" & attraction_coeff_diff == "200-328"').total_revenue
data_3 = df_2.query('group == "A" & attraction_coeff_diff == "329-594"').total_revenue

data_4 = df_2.query('group == "B" & attraction_coeff_diff == "0-199"').total_revenue 
data_5 = df_2.query('group == "B" & attraction_coeff_diff == "200-328"').total_revenue
data_6 = df_2.query('group == "B" & attraction_coeff_diff == "329-594"').total_revenue

In [ ]:
stat, p = kruskal(data_1, data_2, data_3, data_4, data_5, data_6)
print('Statistics=%.3f, p=%.3f' % (stat, p))  # отвергли гипотезу H(0)

In [ ]:
data_att_coef = [data_1, data_2, data_3, data_4, data_5, data_6]

sp.posthoc_dunn(data_att_coef, p_adjust = 'bonferroni')  # perform Dunn's test using a Bonferonni correction for the p-values

In [ ]:
sp.posthoc_dunn(data_att_coef, p_adjust = 'holm')  # perform Dunn's test using a Holm correction for the p-values

In [ ]:

# ВЫВОД: видим, что при использовании непараметрических Kruskal-Wallis test и Dunn test, гипотезы H(0) были отклонены при
# сравнении значений в парах A\200-328 и B\200-328, A\329-594 и B\200-328, что позволяет нам говорить о статистически значимых
# различиях (по метрике 'total_revenue') между пользователями из тестовой группы В и контрольной A1+A2. 

# И с поправкой Бонферрони, и с более "мягкой" поправкой Холма получаем одинаковые конечные результаты.

# Принимаем решение в конечных выводах опираться на "непараметрику" (учитывая более строгий характер поправки Бонферрони;
# учитывая характеристики исходных групп). 


In [ ]:
#                               3.2.5. Фактор "frequency" (частота посещений сайта)

In [ ]:
all_groups['frequency'] = (all_groups['visit_days'].str.count(',') + 1)  # создаем новую колонку "frequency"

In [ ]:
all_groups = all_groups.fillna({'frequency': all_groups.frequency.median()})  # убираем NaN в 'frequency'

In [ ]:
df_3 = all_groups.groupby(['group', 'frequency'], as_index=False).agg({'total_revenue': 'mean'})

In [ ]:
cut_labels_3 = ['1-10', '11-20', '21-31']
cut_bins_3 = [0, 10, 20, 31]

df_3 = df_3.assign(frequency_diff = pd.cut(df_3['frequency'], bins=cut_bins_3, labels=cut_labels_3))

In [ ]:
df_3 = df_3.drop(columns='frequency')

In [ ]:
formula = 'total_revenue ~ group + frequency_diff + group:frequency_diff'
model = ols(formula, df_3).fit()
aov_table = anova_lm(model, typ=2)
print(aov_table)

In [ ]:

# ВЫВОД:  Как видим, эффект от влияния обоих факторов на ЗП ('total_revenue') статистически незначим (как и их взаимодействие) 


In [ ]:
# Проверим, что получим при использовании непараметрических тестов (Kruskal-Wallis Test)

data_3_1 = df_3.query('group == "A" & frequency_diff == "1-10"').total_revenue 
data_3_2 = df_3.query('group == "A" & frequency_diff == "11-20"').total_revenue
data_3_3 = df_3.query('group == "A" & frequency_diff == "21-31"').total_revenue

data_3_4 = df_3.query('group == "B" & frequency_diff == "1-10"').total_revenue 
data_3_5 = df_3.query('group == "B" & frequency_diff == "11-20"').total_revenue
data_3_6 = df_3.query('group == "B" & frequency_diff == "21-31"').total_revenue

In [ ]:
stat, p = kruskal(data_3_1, data_3_2, data_3_3, data_3_4, data_3_5, data_3_6)
print('Statistics=%.3f, p=%.3f' % (stat, p)) # не можем отвергнуть гипотезу H(0)

In [ ]:

# ВЫВОД: при использовании непараметрического Kruskal-Wallis test не можем отвергнуть гипотезу H(0),
# нет статистически значимых различий (по метрике 'total_revenue') между пользователями из тестовой группы В и контрольной A1+A2.


In [ ]:
#                                             4. ОБЩИЕ ВЫВОДЫ ПО ПРОЕКТУ

In [ ]:

# 1.   Можем констатировать, что эксперимент в целом был успешен. Тем не менее, есть вопросы к чистоте планирования
#  эксперимента (некорректное сплитование на группы), что может поставить под вопрос ценность полученных данных.
#      Проанализировав некоторыё основные метрики (Revenue, ARPPU, Active Users) можно увидеть, что за период эксперимента
#  в тестовой группе "В" размер Revenue стал несколько меньше, чем в группе "А1", но больше чем в "А2";  
#  количество Active Users в тестовой группе "В" упало по сравнению с "А1" и "А2" (можно предположить, что они ушли с
#  премиум-подписки в связи с изменившимися тарифами); в то же время за счет изменения стоимости премиум-подписки и
#  сокращения числа пользователей, в тестовой группе "В", по сравнению с группами "А1" и "А2", ощутимо выросла метрика ARPPU. 


In [ ]:

# 2.  Были получены статистически значимые результаты, позволяющие утверждать,что нововведение имеет смысл среди
#  следующих конкретных групп пользователей:   
#     - пользователи возрастной категории 16-20 лет, 56-99 лет;
#     - пользователи из United States of America;
#     - пользователи с показателем "views_count" (от 113 до 364 единиц);
#     - пользователи с показателем "attraction_coeff" (от 200 до 328 единиц). 
